In [3]:
#importamos librería
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler

import plotly.express as px

/Users/eueunesmo/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def listarNulos(dataset):
    df_stat = dataset.isnull().sum()[(dataset.isnull().sum()>0)].sort_values(ascending=False).to_frame()
    df_stat.rename(columns={0: 'Nulos'}, inplace = True)
    percent = (df_stat["Nulos"]/dataset.shape[0]).mul(100)
    df_stat["percentaje"] = percent.round(2)
    return df_stat

#target= ["PRECIO"]
def obtener_lista_variables(dataset):
    lista_numericas=[]
    lista_boolean=[]
    lista_categoricas=[]
    for i in dataset:
        if    (dataset[i].dtype.kind=="f" or dataset[i].dtype.kind=="i") and len(dataset[i].unique())!= 2  and i not in target:
              lista_numericas.append(i)
        elif  (dataset[i].dtype.kind=="f" or dataset[i].dtype.kind=="i")  and len(dataset[i].unique())== 2  and i not in target:
              lista_boolean.append(i)
        elif  (dataset[i].dtype.kind=="O")  and i not in target:
              lista_categoricas.append(i)
    return lista_numericas, lista_boolean, lista_categoricas


In [5]:
#Ler .CSV
df_lloguer = pd.read_csv("Data/2017_lloguer_preu_trim.csv")
df_poblacio = pd.read_csv("Data/2017_poblacio_exposada_barris_mapa_estrategic_soroll_bcn_long.csv")

 DataFrame (1) : Informació sobre la mitja del lloguer mensual (€/mes) i per superfície (€/m2) de laciutat de Barcelona:

In [10]:
print(df_lloguer.info())
df_lloguer.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Any             584 non-null    int64  
 1   Trimestre       584 non-null    int64  
 2   Codi_Districte  584 non-null    int64  
 3   Nom_Districte   584 non-null    object 
 4   Codi_Barri      584 non-null    int64  
 5   Nom_Barri       584 non-null    object 
 6   Lloguer_mitja   584 non-null    object 
 7   Preu            546 non-null    float64
dtypes: float64(1), int64(4), object(3)
memory usage: 36.6+ KB
None


,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu
0,2017,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),734.99
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Lloguer mitjà mensual (Euros/mes),905.26
2,2017,1,1,Ciutat Vella,3,la Barceloneta,Lloguer mitjà mensual (Euros/mes),722.78
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Lloguer mitjà mensual (Euros/mes),895.28
4,2017,1,2,Eixample,5,el Fort Pienc,Lloguer mitjà mensual (Euros/mes),871.08


Para a simplificação da da dimensão dos conjuntos de dados, será aplicada na tabela Lloguer as seguintes transformações:
1. Manuteção apenas das categorias de média de aluguéis por metro quadrado: Pela média do valor por metro quadrado é possível inferir o valor do preço de aluguel para imóvel de qualquer metragem.
2. Substituição dos registros de preço dos imóveis por bairro de cada trimestre pela média anual;
3. Remoção dos registros sobre ano ("Any"), já que a tabela contém apenas registros obtidos no ano de 2017, e 
4. Remoção dos registros sobre o trimestre ("Trimestre"), já que será utilizada a média do preço obtida na leitura dos 4 trimestres do ano

1. Remoção das categorias de aluguel por média mensal, mantendo apenas aluguel por metro quadrado.

In [15]:
#Copia de seguridad
df_lloguer2 = df_lloguer.copy()

#Borrar registros de Lloguer mensual
df_lloguer2.drop(df_lloguer2[df_lloguer2['Lloguer_mitja'] == "Lloguer mitjà mensual (Euros/mes)"].index, inplace=True)

#Verificación
df_lloguer2[df_lloguer2['Lloguer_mitja'] == "Lloguer mitjà mensual (Euros/mes)"]


,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu


2. Utización de una agupación para crear un nuevo DataFrame con los datos apuntados en los pasos 2,3 y 4:

In [35]:
#Copia de seguridad
df_lloguer3 = df_lloguer2.copy()

#Utización de una agupación para crear un nuevo DataFrame con los datos apuntados en los pasos 2,3 y 4:
df_lloguer3_clean = df_lloguer3.groupby(["Codi_Districte","Nom_Districte","Codi_Barri","Nom_Barri"],as_index=False)[["Preu"]].mean()



DataFrame (2) : informació sobre l’exposició als sorolls de la població, del Mapa Estratègic de Soroll de la ciutat de Barcelona:

In [36]:
print(df_poblacio.info())
df_poblacio.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18980 entries, 0 to 18979
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codi_Districte  18980 non-null  int64 
 1   Nom_Districte   18980 non-null  object
 2   Codi_Barri      18980 non-null  int64 
 3   Nom_Barri       18980 non-null  object
 4   Concepte        18980 non-null  object
 5   Rang_soroll     18980 non-null  object
 6   Valor           18980 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.0+ MB
None


,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Concepte,Rang_soroll,Valor
0,1,Ciutat Vella,1,el Raval,TOTAL_D,<40 dB,7.73%
1,1,Ciutat Vella,1,el Raval,TOTAL_D,40-45 dB,26.98%
2,1,Ciutat Vella,1,el Raval,TOTAL_D,45-50 dB,7.38%
3,1,Ciutat Vella,1,el Raval,TOTAL_D,50-55 dB,11.97%
4,1,Ciutat Vella,1,el Raval,TOTAL_D,55-60 dB,19.85%


Para a simplificação da da dimensão dos conjuntos de dados, será aplicada na tabela Lloguer as seguintes transformações:
1. Manuteção apenas dos registros de conceitos ("CONCEPTE") equivalentes a "TOTAL_DEN", já que, segundo manual disponível na referência do site, diz respeito ao nível ponderado de ruído para os 3 horários. 
2. Criação da variável "RANG_SOROLL_CLASE". Essa variável será composta pelos valores 1, 2 e 3 , a depender da distribuição dos percentuais dos nívels de ruído ("RANG_SOROLL"): 1: [<40 dB -> 50 dB[ ; 2: " [50 db -> 65 dB); 3: [65dB --> >=80 dB]

In [38]:
#Copia de Seguridad
df_poblacio2 = df_poblacio.copy()

#Borrar registros de Concepte != "TOTAL_DEN"
df_poblacio2.drop(df_poblacio2[df_poblacio2['Concepte'] != "TOTAL_DEN"].index, inplace=True)

#Verificación
df_poblacio2[df_poblacio2['Concepte'] != "TOTAL_DEN"]

#df_poblacio[df_poblacio["Nom_Barri"]=="el Raval"].head(100)

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Concepte,Rang_soroll,Valor


In [83]:
#Copia de Seguridad
df_poblacio3 = df_poblacio2.copy()

In [84]:
#Conversión del "Valor" de % para float
df_poblacio3["Valor"] = df_poblacio3["Valor"].str.replace("%","")
df_poblacio3["Valor"] = df_poblacio3["Valor"].astype("float")

In [106]:
#Definir lista de bairros
lista_barri = df_poblacio3["Nom_Barri"].unique()
lista_barri

#División por rangos
rango1 = ['<40 dB', '40-45 dB', '45-50 dB']
rango2 = ['50-55 dB', '55-60 dB','60-65 dB']
rango3 = ['65-70 dB', '70-75 dB', '75-80 dB', '>=80 dB']

#Diccionário "Barrio" -  "Nível ruído"
dic_rango={}

for n in lista_barri:
    grupo = df_poblacio3[["Rang_soroll","Valor"]][df_poblacio3["Nom_Barri"]==n]
    sum_rango1=0
    sum_rango2=0
    sum_rango3=0
      
    for x in range(grupo.shape[0]):
        if(grupo.iloc[x,0] in rango1 ):
            sum_rango1 += grupo.iloc[x,1]
        elif(grupo.iloc[x,0] in rango2 ):
            sum_rango2 += grupo.iloc[x,1]
        elif(grupo.iloc[x,0] in rango3 ):
            sum_rango3 += grupo.iloc[x,1]
    
    if(max(sum_rango1,sum_rango2,sum_rango1)==sum_rango1):
        dic_rango[n]=1
    elif(max(sum_rango1,sum_rango2,sum_rango1)==sum_rango2):
        dic_rango[n]=2
    elif(max(sum_rango1,sum_rango2,sum_rango1)==sum_rango3):
        dic_rango[n]=3
    
#verificacion
print(dic_rango)

#Definición del Dataframe reducido
df_poblacio_clean = pd.DataFrame.from_dict(dic_rango,orient='index', columns=["Nivel_Ruído"])
df_poblacio_clean["Nom_Barri"] = df_poblacio_clean.index
df_poblacio_clean.index = range(0,df_poblacio_clean.shape[0])
df_poblacio_clean

{'el Raval': 1, 'el Barri Gòtic': 2, 'la Barceloneta': 2, 'Sant Pere, Santa Caterina i la Ribera': 2, 'el Fort Pienc': 1, 'la Sagrada Família': 1, "la Dreta de l'Eixample": 2, "l'Antiga Esquerra de l'Eixample": 2, "la Nova Esquerra de l'Eixample": 1, 'Sant Antoni': 1, 'el Poble Sec': 1, 'la Marina del Prat Vermell': 2, 'la Marina de Port': 2, 'la Font de la Guatlla': 2, 'Hostafrancs': 2, 'la Bordeta': 2, 'Sants - Badal': 1, 'Sants': 1, 'les Corts': 2, 'la Maternitat i Sant Ramon': 2, 'Pedralbes': 2, 'Vallvidrera, el Tibidabo i les Planes': 1, 'Sarrià': 2, 'les Tres Torres': 2, 'Sant Gervasi - la Bonanova': 2, 'Sant Gervasi - Galvany': 2, 'el Putxet i el Farró': 2, 'Vallcarca i els Penitents': 2, 'el Coll': 2, 'la Salut': 2, 'la Vila de Gràcia': 2, "el Camp d'en Grassot i Gràcia Nova": 2, 'el Baix Guinardó': 1, 'Can Baró': 2, 'el Guinardó': 2, "la Font d'en Fargues": 2, 'el Carmel': 1, 'la Teixonera': 2, 'Sant Genís dels Agudells': 2, 'Montbau': 2, "la Vall d'Hebron": 2, 'la Clota': 2, 

,Nivel_Ruído,Nom_Barri
0,1,el Raval
1,2,el Barri Gòtic
2,2,la Barceloneta
3,2,"Sant Pere, Santa Caterina i la Ribera"
4,1,el Fort Pienc
...,...,...
68,2,Diagonal Mar i el Front Marítim del Poblenou
69,2,el Besòs i el Maresme
70,2,Provençals del Poblenou
71,2,Sant Martí de Provençals


In [129]:
df_pb = df_poblacio.copy()

df_pb["Valor"] = df_pb["Valor"].str.replace("%","")
df_pb["Valor"] = df_pb["Valor"].astype("float")
#df_pb["Concepte"].value_counts().sort_values(ascending=True)
#df_pb["Valor"] = df_pb["Valor"]/100
df_pb[(df_pb["Nom_Barri"]=="el Raval") & (df_pb["Concepte"]=="VIANANTS_D")].groupby(["Nom_Barri", "Concepte","Rang_soroll"])["Valor"].sum()
#df_poblacio.groupby(["Nom_Barri", "Concepte", "Rang_soroll"])["Valor"].
#(df_accidents[df_accidents["Nom_barri"]!="Desconegut"]).count()
#(df_accidents["Nom_barri"]).value_counts()
#df_pb[(df_pb["Concepte"]=="TOTAL_DEN")].groupby(["Nom_Barri", "Concepte"])["Valor"].sum()

Nom_Barri  Concepte    Rang_soroll
el Raval   VIANANTS_D  40-45 dB       13.35
                       45-50 dB       10.90
                       50-55 dB       10.81
                       55-60 dB        6.47
                       60-65 dB        0.78
                       65-70 dB        0.00
                       70-75 dB        0.00
                       75-80 dB        0.00
                       <40 dB         57.69
                       >=80 dB         0.00
Name: Valor, dtype: float64

In [108]:
df_lloguer3_clean.set_index('Nom_Barri').join(df_poblacio_clean.set_index('Nom_Barri'))

,Codi_Districte,Nom_Districte,Codi_Barri,Preu,Nivel_Ruído
Nom_Barri,,,,,
el Raval,1,Ciutat Vella,1,13.8950,1
el Barri Gòtic,1,Ciutat Vella,2,14.1900,2
la Barceloneta,1,Ciutat Vella,3,19.8450,2
"Sant Pere, Santa Caterina i la Ribera",1,Ciutat Vella,4,15.7050,2
el Fort Pienc,2,Eixample,5,13.0850,1
...,...,...,...,...,...
Diagonal Mar i el Front Marítim del Poblenou,10,Sant Martí,69,16.0575,2
el Besòs i el Maresme,10,Sant Martí,70,9.9675,2
Provençals del Poblenou,10,Sant Martí,71,12.9825,2
